In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import numpy as np
import pandas as pd
from time import sleep
from random import randint
import datetime

In [2]:
# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
city = 'Chandler'

# Create empty list to store scraped data
datatoget = []

for i in range(0,8):

    # Set url
    url = f'https://phoenix.craigslist.org/search/chandler-az/apa#search=1~thumb~{i}~0'
    browser.visit(url)
    sleep(randint(5,7))

    # Parse the HTML
    html = browser.html
    soup = bs(html, 'html.parser')

    # Find and extract relevant data
    search = soup.find_all('div', class_='result-info')

    print('\n================')
    print(f'This is page {i+1}')
    print('================')    

    # Loop through data found and extract relevant information
    for (counter,link) in enumerate(search):

        # Scrape dates
        try:
            dates = link.find_all('div', class_='meta')[0].text.split('·')[0]

            current_date = datetime.datetime.now()

            if link.find_all('div', class_='meta')[0].text.split('·')[0][-9:] == 'hours ago':
                dates = current_date.strftime(f'{current_date.month}/{current_date.day}')

            elif link.find_all('div', class_='meta')[0].text.split('·')[0][-11:] == 'minutes ago':
                dates = current_date.strftime(f'{current_date.month}/{current_date.day}')
            
            elif link.find_all('div', class_='meta')[0].text.split('·')[0][-8:] == 'hour ago':
                dates = current_date.strftime(f'{current_date.month}/{current_date.day}')

        except:
            dates = np.nan

        # Scrape titles
        titles = link.find_all('a', class_='titlestring')[0].text

        # Scrape links
        links = link.find_all('a', class_='titlestring', href=True)[0]['href']

        try:
        # Scrape prices
            prices = link.find_all('span', class_='priceinfo')[0].text
        except:
            prices = np.nan

        # Scrape neighborhoods and handle any errors if no neighborhood found   
        try:
            neighborhoods = link.find('div', class_='supertitle').text

        except:
            neighborhoods = np.nan

        # Visit each link seperately
        browser.visit(links)

        # Ensures that the server does not get overloaded with requests
        sleep(randint(2,5))  

        # After visiting link, extract bath and amenities data
        html = browser.html
        soup = bs(html, 'html.parser')
        
        # Error handeling in case there are no sqft displayed
        try:
            if soup.find_all('p', class_='attrgroup')[0].text.split('\n')[2][-3:] == 'ft2':
                sqft = soup.find_all('p', class_='attrgroup')[0].text.split('\n')[2][:-3]
            else:
                sqft = np.nan
        except:
            sqft = np.nan
        
        # Error handeling in case there are no bedrooms displayed
        try:
            bedrooms = soup.find_all('p', class_='attrgroup')[0].text.split('\n')[1].split('/')[0].strip().lower()

        except:
            bedrooms = np.nan
        
        try:
            baths = soup.find_all('p', class_='attrgroup')[0].text.split('\n')[1].split()[2][:-2]

        except:
            baths = np.nan


        amenities = soup.find_all('p', class_='attrgroup')[1].text.split('\n')
        # List comprehension used to make sure no empty strings get added to the list
        all_amenities = [am for am in amenities if am != '']

        # Print statements to ensure that the correct data is scraped
        print(f'\nListing Number: {counter+1}')
        print(f'Square Footage: {sqft if sqft == sqft else "not available"}')
        print(f'Price: {prices if prices == prices else "not available"}')
        print(f'Title: {titles}')
        print(f'Date: {dates}')
        print(f'Number of Bedrooms: {bedrooms if bedrooms == bedrooms else "not available"}')
        print(f'Neighborhood: {neighborhoods if neighborhoods == neighborhoods else "not available"}')
        print(f'Number of Bathrooms: {baths if baths == baths else "not available"}')
        print(f'Link: {links}')
        print(f'Amenities: {all_amenities}')

        # Create dicitonary for our DataFrame
        data = {'date': dates,
                'title': titles,
                'link': links,
                'price': prices,
                'bedroom': bedrooms,
                'sqft': sqft,
                'neighborhood': neighborhoods,
                'bathroom': baths,
                'amenities': all_amenities}

        # Append the data
        datatoget.append(data)

print('\n================ Done ================')

# Quit automated browser instance
browser.quit()


This is page 1

Listing Number: 1
Square Footage: 782
Price: $1,265
Title: Resident Rewards Program, Ceramic Tiled Baths, Two Spas
Date: 1/17
Number of Bedrooms: 1br
Neighborhood: 843 South Longmore Street, Mesa, AZ
Number of Bathrooms: 1
Link: https://phoenix.craigslist.org/evl/apa/d/mesa-resident-rewards-program-ceramic/7571390055.html
Amenities: ['air conditioning', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'carport', 'rent period: monthly', 'wheelchair accessible']

Listing Number: 2
Square Footage: 811
Price: $1,446
Title: Washer & Dryer, Detached Garage, Upgraded Interior
Date: 1/17
Number of Bedrooms: 2br
Neighborhood: west valley
Number of Bathrooms: 1
Link: https://phoenix.craigslist.org/wvl/apa/d/tempe-washer-dryer-detached-garage/7570273780.html
Amenities: ['apartment', 'w/d in unit', 'detached garage', 'rent period: monthly']

Listing Number: 3
Square Footage: 720
Price: $1,439
Title: BBQ/picnic area
Date: 1/17
Number of Bedrooms: 1br
Neighb


Listing Number: 21
Square Footage: 1134
Price: $1,775
Title: Sweet Luxury! 2 bedroom apt! 50% off 3 first 3 months! Call now!
Date: 1/17
Number of Bedrooms: 2br
Neighborhood: 1930 S 24th St, Phoenix, AZ
Number of Bathrooms: 2
Link: https://phoenix.craigslist.org/evl/apa/d/mesa-sweet-luxury-bedroom-apt-50-off/7577942957.html
Amenities: ['open house dates', 'friday 2023-01-13', 'saturday 2023-01-14']

Listing Number: 22
Square Footage: 983
Price: $1,605
Title: Faux Wood Flooring, Clubhouse with Free Wi-Fi, Fireplace
Date: 1/17
Number of Bedrooms: 2br
Neighborhood: east valley
Number of Bathrooms: 2
Link: https://phoenix.craigslist.org/evl/apa/d/tempe-faux-wood-flooring-clubhouse-with/7577879370.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'carport', 'rent period: monthly']

Listing Number: 23
Square Footage: 792
Price: $1,540
Title: Crown molding
Date: 1/17
Number of Bedrooms: 1br
Neighborhood: east valley
Number of Bathrooms: 1
Link: https:


Listing Number: 42
Square Footage: 800
Price: $1,739
Title: Spacious 1 Bedroom Renovated Apartment Home
Date: 1/16
Number of Bedrooms: 1br
Neighborhood: 15815 S Lakewood Pkwy W
Number of Bathrooms: 1
Link: https://phoenix.craigslist.org/cph/apa/d/phoenix-spacious-bedroom-renovated/7575687737.html
Amenities: ['air conditioning', 'application fee details: Application Fee: $50 (Anyone applying that is 18 or older).', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'no smoking', 'carport', 'rent period: monthly']

Listing Number: 43
Square Footage: 1152
Price: $1,570
Title: Elevate at Discovery park beatiful apartments
Date: 1/16
Number of Bedrooms: 2br
Neighborhood: east valley
Number of Bathrooms: 2
Link: https://phoenix.craigslist.org/evl/apa/d/tempe-elevate-at-discovery-park/7575533318.html
Amenities: ['apartment', 'w/d in unit', 'detached garage', 'rent period: monthly']

Listing Number: 44
Square Footage: 1121
Price: $1,730
Title: 2BD 2BA, Furnished apartme


Listing Number: 61
Square Footage: not available
Price: $1,695
Title: We LOVE OUR VETERANS - RENT-TO-OWN/BUY/OR RENT TODAY!
Date: 1/16
Number of Bedrooms: 2br
Neighborhood: East Valley
Number of Bathrooms: 2
Link: https://phoenix.craigslist.org/evl/apa/d/we-love-our-veterans-rent-to-own-buy-or/7577860343.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'off-street parking', 'rent period: monthly']

Listing Number: 62
Square Footage: 900
Price: $1,785
Title: YAY - KEEP YOUR PET!! ***LOW DEPOSITS ***Call Apartment Options!!***
Date: 1/16
Number of Bedrooms: 2br
Neighborhood: Mesa / Tempe / Chandler / Ahwatukee
Number of Bathrooms: 2
Link: https://phoenix.craigslist.org/evl/apa/d/mesa-yay-keep-your-pet-low-deposits/7570834829.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'condo', 'w/d in unit', 'off-street parking', 'rent period: monthly', 'wheelchair accessible']

Listing Number: 63
Square Footage: not available
Price: $1,595
Ti


Listing Number: 81
Square Footage: 701
Price: $1,295
Title: Fitness Center, Pet Friendly Community, Picnic Area with Grill
Date: 1/16
Number of Bedrooms: 1br
Neighborhood: Mesa
Number of Bathrooms: 1
Link: https://phoenix.craigslist.org/evl/apa/d/mesa-fitness-center-pet-friendly/7579300130.html
Amenities: ['air conditioning', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'no smoking', 'carport', 'rent period: monthly', 'wheelchair accessible']

Listing Number: 82
Square Footage: 1006
Price: $1,550
Title: 2 Clubhouses, 4 Playgrounds, 1 Roller Hockey Area, Upgraded Kitchens*
Date: 1/16
Number of Bedrooms: 2br
Neighborhood: Gilbert
Number of Bathrooms: 2
Link: https://phoenix.craigslist.org/evl/apa/d/mesa-clubhouses-playgrounds-roller/7579300099.html
Amenities: ['air conditioning', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'carport', 'rent period: monthly', 'wheelchair accessible']

Listing Number: 83
Square Footage: 660
Price: 

...


Listing Number: 27
Square Footage: not available
Price: $2,199
Title: $1000 off first full month, Resort style apartment homes by the lake!
Date: 1/14
Number of Bedrooms: 3br
Neighborhood: Gilbert, off the 60
Number of Bathrooms: 2
Link: https://phoenix.craigslist.org/evl/apa/d/gilbert-1000-off-first-full-month/7576235411.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'carport', 'rent period: monthly']

Listing Number: 28
Square Footage: not available
Price: $2,199
Title: $1000 in free rent, Resort style apartment homes by the lake!
Date: 1/14
Number of Bedrooms: 3br
Neighborhood: Gilbert, For Immediate Move in
Number of Bathrooms: 2
Link: https://phoenix.craigslist.org/evl/apa/d/gilbert-1000-in-free-rent-resort-style/7577386545.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'carport', 'rent period: monthly']

Listing Number: 29
Square Footage: 1054
Price: $1,544
Title: Move-in with NO SECURITY DEP


Listing Number: 46
Square Footage: 1063
Price: $1,977
Title: Limited Access Gates, Whirlpool Spa, Breakfast Bars
Date: 1/14
Number of Bedrooms: 2br
Neighborhood: Chandler
Number of Bathrooms: 2
Link: https://phoenix.craigslist.org/evl/apa/d/tempe-limited-access-gates-whirlpool/7578531245.html
Amenities: ['air conditioning', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'no smoking', 'carport', 'rent period: monthly']

Listing Number: 47
Square Footage: 395
Price: $389
Title: ONE BEDROOM APTS - NO LEASES
Date: 1/14
Number of Bedrooms: 1br
Neighborhood: Mesa
Number of Bathrooms: 1
Link: https://phoenix.craigslist.org/evl/apa/d/mesa-one-bedroom-apts-no-leases/7578504130.html
Amenities: ['air conditioning', 'furnished', 'apartment', 'laundry on site', 'off-street parking', 'rent period: weekly']

Listing Number: 48
Square Footage: 670
Price: $1,539
Title: Great Gilbert Neighborhood and Schools, Onsite Management
Date: 1/14
Number of Bedrooms: 1br
Neighborhood: 


Listing Number: 65
Square Footage: 395
Price: $389
Title: 1 Bedroom Apartment -No Leases
Date: 1/14
Number of Bedrooms: 1br
Neighborhood: Mesa
Number of Bathrooms: 1
Link: https://phoenix.craigslist.org/evl/apa/d/mesa-bedroom-apartment-no-leases/7578484046.html
Amenities: ['air conditioning', 'furnished', 'apartment', 'laundry on site', 'off-street parking', 'rent period: weekly']

Listing Number: 66
Square Footage: 1000
Price: $1,640
Title: Lease Today, Two Pools and Spa, Resort Style Pools
Date: 1/14
Number of Bedrooms: 2br
Neighborhood: Phoenix
Number of Bathrooms: 2
Link: https://phoenix.craigslist.org/evl/apa/d/phoenix-lease-today-two-pools-and-spa/7578512534.html
Amenities: ['air conditioning', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'no smoking', 'carport', 'rent period: monthly', 'wheelchair accessible']

Listing Number: 67
Square Footage: 395
Price: $389
Title: 1 Bedroom Apartment -No Leases
Date: 1/14
Number of Bedrooms: 1br
Neighborhood: Me


Listing Number: 84
Square Footage: 870
Price: $2,260
Title: Rental in Chandler with Right to Purchase
Date: 1/14
Number of Bedrooms: 2br
Neighborhood: Sunstone
Number of Bathrooms: 1.5
Link: https://phoenix.craigslist.org/evl/apa/d/chandler-rental-in-chandler-with-right/7576775460.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'townhouse', 'w/d in unit', 'carport', 'rent period: monthly']

Listing Number: 85
Square Footage: 1856
Price: $2,800
Title: Rental in Chandler with Right to Purchase
Date: 1/14
Number of Bedrooms: 3br
Neighborhood: Tre Vincino
Number of Bathrooms: 2.5
Link: https://phoenix.craigslist.org/evl/apa/d/chandler-rental-in-chandler-with-right/7576779199.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'house', 'w/d hookups', 'attached garage', 'rent period: monthly']

Listing Number: 86
Square Footage: 1144
Price: $1,625
Title: Refrigerator, Wi-Fi at Pool and Clubhouse, Large Open Green Spaces
Date: 1/14
Number of Bedrooms: 2br
Neighborho


Listing Number: 103
Square Footage: 745
Price: $1,544
Title: In-Home Full Size Washer & Dryer, Private Balcony or Patio
Date: 1/14
Number of Bedrooms: 1br
Neighborhood: The islands in Gilbert
Number of Bathrooms: 1
Link: https://phoenix.craigslist.org/evl/apa/d/gilbert-in-home-full-size-washer-dryer/7578416180.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'carport', 'rent period: monthly', 'wheelchair accessible']

Listing Number: 104
Square Footage: 1054
Price: $1,588
Title: Move-in with NO SECURITY DEPOSIT: Only a small one-time payment
Date: 1/14
Number of Bedrooms: 2br
Neighborhood: Chandler
Number of Bathrooms: 2
Link: https://phoenix.craigslist.org/evl/apa/d/tempe-move-in-with-no-security-deposit/7574101942.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'carport', 'rent period: monthly']

Listing Number: 105
Square Footage: 782
Price: $1,265
Title: Vertical Blinds, On-line Maintenance Reques

In [4]:
# Creating DataFrame
df = pd.DataFrame(datatoget)
df

,date,title,link,price,bedroom,sqft,neighborhood,bathroom,amenities
0,1/17,"Resident Rewards Program, Ceramic Tiled Baths,...",https://phoenix.craigslist.org/evl/apa/d/mesa-...,"$1,265",1br,782,"843 South Longmore Street, Mesa, AZ",1,"[air conditioning, cats are OK - purrr, dogs a..."
1,1/17,"Washer & Dryer, Detached Garage, Upgraded Inte...",https://phoenix.craigslist.org/wvl/apa/d/tempe...,"$1,446",2br,811,west valley,1,"[apartment, w/d in unit, detached garage, rent..."
2,1/17,BBQ/picnic area,https://phoenix.craigslist.org/evl/apa/d/tempe...,"$1,439",1br,720,east valley,1,"[apartment, w/d in unit, carport, rent period:..."
3,1/17,"Clubhouse with Free Wi-Fi, Stainless Steel App...",https://phoenix.craigslist.org/evl/apa/d/tempe...,"$1,755",2br,1167,east valley,2,"[cats are OK - purrr, dogs are OK - wooof, apa..."
4,1/17,High Vaulted Ceilings In Your New Beautiful 3 ...,https://phoenix.craigslist.org/cph/apa/d/phoen...,"$1,970",3br,1200,"15815 S LAKEWOOD PKWY W, PHOENIX, AZ",2,"[air conditioning, application fee details: Ap..."
...,...,...,...,...,...,...,...,...,...
955,1/14,1 Bedroom 1 Bath Apartment -No Leases,https://phoenix.craigslist.org/evl/apa/d/mesa-...,$389,1br,395,825 s Dobson Rd,1,"[air conditioning, furnished, apartment, laund..."
956,1/14,ONE BEDROOM APTS - NO CREDIT CHECKS,https://phoenix.craigslist.org/evl/apa/d/mesa-...,$389,1br,395,Mesa,1,"[air conditioning, furnished, apartment, laund..."
957,1/14,ONE BEDROOM APTS - NO CREDIT CHECKS,https://phoenix.craigslist.org/evl/apa/d/mesa-...,$389,1br,395,Mesa,1,"[air conditioning, furnished, apartment, laund..."
958,1/14,1 Bedroom Apartment -No Leases,https://phoenix.craigslist.org/evl/apa/d/mesa-...,$389,1br,395,Mesa,1,"[air conditioning, cats are OK - purrr, dogs a..."


In [5]:
# Exporting data to csv
df.to_csv(f"{city}_data_raw.csv", index=False)